## Configuration

### Path variables

In [0]:
#@title Set colab path in Google Drive

#@markdown Path to the folder where colab should read data from and write to
GDRIVE_COLAB_PATH       = "colab"           # @param {type: "string"}

In [0]:
#@title Set experiment group and experiment name
#@markdown The output path for this experiment is determined by the following two variables.

#@markdown Name of the experiment group this experiment belongs to
EXPERIMENT_GROUP        = "ResNet-Optimizer"   # @param {type: "string"}

#@markdown Name of this experiment
EXPERIMENT_NAME         = "ResNet-SGD"      # @param {type: "string"}

In [0]:
import os

# basic paths
GDRIVE_MOUNTING_PATH    = "/gdrive"                                             # path where the google drive will be mounted
                                                                                # remote path of colab data
REMOTE_DATA_PATH        = os.path.join(GDRIVE_MOUNTING_PATH, "My Drive", GDRIVE_COLAB_PATH)  
LOCAL_DATA_PATH         = "/home/data"                                          # local path of colab data
REMOTE_ROBOCUP          = os.path.join(REMOTE_DATA_PATH, "Robo Cup")            # remote path of robo cup data
LOCAL_ROBOCUP           = os.path.join(LOCAL_DATA_PATH,  "Robo Cup")            # local path of robo cup data

# required paths for data mangling
TRAIN_PATH_UNSORTED     = os.path.join(LOCAL_ROBOCUP, "train")                  # path of train data before data reorganization
TEST_PATH_UNSORTED      = os.path.join(LOCAL_ROBOCUP, "test")                   # path of test data before data reorganization

# required paths for training
TRAIN_PATH_SORTED       = os.path.join(LOCAL_ROBOCUP, "train_sorted")           # path of train data after data reorganization
TEST_PATH_SORTED        = os.path.join(LOCAL_ROBOCUP, "test_sorted")            # path of test data after data reorganization

REMOTE_OUTPUT_PATH      = os.path.join(REMOTE_DATA_PATH, "output")              # path where output of colab is written to or read from
                                                                                # used for saving the training session, model and analysis
                                                                            
                                                                                # path where the output of the experiment is written to
REMOTE_OUTPUT_EXPERIMENT_PATH = os.path.join(REMOTE_OUTPUT_PATH, EXPERIMENT_GROUP, EXPERIMENT_NAME)                             
                                                                            
                                                                                # path where the analysis results are written to
REMOTE_OUTPUT_ANALYSIS_PATH   = os.path.join(REMOTE_OUTPUT_EXPERIMENT_PATH, "analysis")
                        


### Path configuration for training

In [0]:
PATH_CONFIG = PathConfig(
    model_path   = REMOTE_OUTPUT_EXPERIMENT_PATH,
    model_file   = "RoboCup-model.hdf5", 
    session_path = REMOTE_OUTPUT_EXPERIMENT_PATH, 
    session_file = "session.pickle",
    train_path   = TRAIN_PATH_SORTED, 
    test_path    = TEST_PATH_SORTED)

### Training mode

In [0]:
TRAINING_MODE = TrainingMode.TRY_RESUME_TRAINING

### Training config

In [0]:
import tensorflow as tf

TRAINING_CONFIG = TrainingConfig(
    optimizer        = tf.keras.optimizers.SGD(), 
    loss_function    = tf.keras.losses.categorical_crossentropy, 
    metrics          = [tf.keras.metrics.Accuracy(), tf.keras.metrics.CategoricalAccuracy()],
    train_batch_size = 10, 
    test_batch_size  = 10, 
    epochs           = 1, 
    super_epochs     = 50 )

### Model provider

In [0]:
class ModelProvider(AbstractModelProvider):
  def _create_model(self):
    print("Creating new model")
    # get model with pre trained weights
    model = tf.keras.applications.ResNet50(weights="imagenet")
    # replace last softmax layer from ImageNet domain with new soft max layer for our RoboCup domain 
    x = tf.keras.layers.Dense(8, activation='softmax', name='predictions')(model.layers[-2].output)
    # create new model
    return tf.keras.models.Model(inputs=model.input, outputs=x)

MODEL_PROVIDER = ModelProvider()